<a href="https://colab.research.google.com/github/plotsaf8/first/blob/main/%D0%91%D0%B5%D1%80%D0%B5%D0%B7%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F03_HWdone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Задачи к Лекции 3

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("/content/mlbootcamp5_train.csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Классы в Python

Нередко, возникает необходимость создания объектов с каким-нибудь внутренним поведением и состоянием. Примерами таких объектов являются классификаторы sklearn, массивы numpy и много другое. Такой объект можно объявить с помощью ключевого слова **class**

```python
class SomeObject:
    def __init__(self, depth):
        self.a = depth
        self.target = None
        
    def fit(self, data, target):
        self.target = data
        # magic
        return
    
    def predict(self, data):
        return self.target    
```

После этого в коде можно будет создать экземпляр данного класса
```python
a = SomeObject(depth=5)
a.fit(data, target)
a.predict(data)
```

## Задачи

**1. В sklearn на данный момент отсутствует функционал для построения деревьев решений из категориальных данных, поэтому его нужно сделать самостоятельно и проверить его работу. Что нужно сделать:**

* __создать классификатор используя только pandas, numpy и scipy. Необходимо его сделать самому, используя исключительно только numpy, pandas и scipy (запрещено использовать sklearn и прочие библиотеки). Напоминаю, что для категориальных данных операция < или > не имеют смысла (использовать только != и ==). Гиперпараметром данного классификатора должна быть максимальная глубина дерева.__
* __Проверить работу данного классификатора на наборе ("gender", "cholesterol", "gluc").__
* __С помощью кросс-валидации найти оптимальную глубину этого дерева. Для вашего классификатора GridSearchCV не подойдет, придется это сделать также самостоятельно.__
* __Нарисовать полученное дерево (я должен понять, как и откуда вы его нарисовали).__

Алгоритм работы классификатора:
 1. Перебираем все возможные признаки и смотрим либо неопределенность Джини, либо прирост информации. Это даст критерий разбиения в виде "признак == значение"
 2. Если выборка полученная при разбиении состоит из объектов одного класса (соответсвует нулевой энтропии), то данный лист просто возвращает значение этого класса.
 3. В противном случае, образуется новый узел и для него начинаем с пункта 1.
 4. Если достигли максимальной глубины, то вместа узла создаем лист, который возвращает самое вероятное значение.

__Замечание:__ в этой задаче не нужно использовать onehot-кодирование.

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

def gini_impurity(y):
    if len(y) == 0:
        return 0
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities**2)

class DecisionTree:
    def __init__(self, max_depth=5):
        self.max_depth = max_depth
        self.tree = {}

    def _find_best_split(self, X, y):
        best_gini = float('inf')
        best_feature = None
        best_value = None

        for feature in X.columns:
            unique_values = np.unique(X[feature])
            for value in unique_values:
                left_mask = X[feature] == value
                right_mask = ~left_mask

                y_left = y[left_mask]
                y_right = y[right_mask]

                if len(y_left) == 0 or len(y_right) == 0:
                    continue

                # Взвешенная неопределенность Джини после разбиения
                gini_left = gini_impurity(y_left)
                gini_right = gini_impurity(y_right)
                total_gini = (len(y_left) * gini_left + len(y_right) * gini_right) / len(y)

                if total_gini < best_gini:
                    best_gini = total_gini
                    best_feature = feature
                    best_value = value

        return best_feature, best_value

    def _build_tree(self, X, y, depth=0):
        # Условия остановки
        if depth >= self.max_depth or len(np.unique(y)) == 1:
            return Counter(y).most_common(1)[0][0]  # Возвращаем наиболее частый класс

        # Поиск оптимального разбиения
        feature, value = self._find_best_split(X, y)
        if feature is None:
            return Counter(y).most_common(1)[0][0]

        # Разделение данных
        left_mask = X[feature] == value
        right_mask = ~left_mask

        # Рекурсивное построение поддеревьев
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'feature': feature,
            'value': value,
            'left': left_subtree,
            'right': right_subtree
        }

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _predict_sample(self, sample, tree):
        if isinstance(tree, dict):
            if sample[tree['feature']] == tree['value']:
                return self._predict_sample(sample, tree['left'])
            else:
                return self._predict_sample(sample, tree['right'])
        else:
            return tree  # Листовой узел (класс)

    def predict(self, X):
        return np.array([self._predict_sample(row, self.tree) for _, row in X.iterrows()])

# Загрузка данных
df = pd.read_csv("/content/mlbootcamp5_train.csv", sep=";", index_col="id")
features = df[["gender", "cholesterol", "gluc"]]
target = df["cardio"]

from sklearn.model_selection import KFold

# Параметры кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)
depths = range(1, 11)
accuracy_scores = []

for depth in depths:
    fold_accuracies = []
    for train_idx, val_idx in kf.split(features):
        X_train, X_val = features.iloc[train_idx], features.iloc[val_idx]
        y_train, y_val = target.iloc[train_idx], target.iloc[val_idx]

        model = DecisionTree(max_depth=depth)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        accuracy = np.mean(preds == y_val)
        fold_accuracies.append(accuracy)

    accuracy_scores.append(np.mean(fold_accuracies))

# Оптимальная глубина
optimal_depth = depths[np.argmax(accuracy_scores)]
print(f"Optimal Depth: {optimal_depth}, Accuracy: {max(accuracy_scores):.4f}")

# Обучение модели с оптимальной глубиной
final_model = DecisionTree(max_depth=optimal_depth)
final_model.fit(features, target)

# Функция для визуализации дерева
def print_tree(tree, indent=""):
    if isinstance(tree, dict):
        print(f"{indent}{tree['feature']} == {tree['value']}?")
        print(f"{indent}├── Yes:", end="")
        print_tree(tree['left'], indent + "│   ")
        print(f"{indent}└── No:", end="")
        print_tree(tree['right'], indent + "    ")
    else:
        print(f" → class {tree}")

# Вывод структуры дерева
print("Decision Tree Structure:")
print_tree(final_model.tree)

Optimal Depth: 2, Accuracy: 0.5918
Decision Tree Structure:
cholesterol == 1?
├── Yes:│   gluc == 2?
│   ├── Yes: → class 1
│   └── No: → class 0
└── No:    cholesterol == 2?
    ├── Yes: → class 1
    └── No: → class 1
